In [1]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import schedule
import pytz
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# =========================
# 🔧 설정값 (여기만 수정)
# =========================
TELEGRAM_TOKEN = "7636841205:AAFScTMxkiNxg9AtGufSOUHS6UTrbwE46tQ"   # BotFather 발급 토큰
CHAT_ID       = "7246158575"       # @userinfobot 또는 getUpdates로 확인한 숫자 ID
SEND_TIME     = "07:50"                # 매일 전송 시각 (한국시간)
NEWS_LIMIT    = 5                      # 기사 개수
SUMMARY_MAXLEN= 200                    # 요약 최대 토큰 길이 (짧게/빠르게 원하면 줄이세요)

# =========================
# 공통 상수/헤더
# =========================
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
}

# =========================
# 텔레그램 도우미 (자동 분할 전송)
# =========================
def send_telegram_message(text: str):
    """
    텔레그램 메시지 길이 제한(약 4096자) 대비 자동 분할 전송.
    """
    max_len = 3800  # 안전 마진
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    if len(text) <= max_len:
        payload = {"chat_id": CHAT_ID, "text": text}
        return requests.post(url, data=payload, timeout=15).json()

    # 길면 분할
    parts = []
    start = 0
    while start < len(text):
        end = min(start + max_len, len(text))
        parts.append(text[start:end])
        start = end

    last_resp = None
    for p in parts:
        payload = {"chat_id": CHAT_ID, "text": p}
        last_resp = requests.post(url, data=payload, timeout=15).json()
        time.sleep(0.5)  # 과도 호출 방지
    return last_resp

# =========================
# 네이버 경제 뉴스: 헤드라인 수집 (다중 선택자 + 폴백)
# =========================
def _extract_articles(soup):
    """
    네이버 섹션 구조가 바뀌어도 견딜 수 있게 다중 선택자 사용.
    반환: [(title, href), ...]
    """
    candidates = []

    # 구형 섹션 UI
    for a in soup.select("div.cluster_body a.cluster_text_headline"):
        t, h = a.get_text(strip=True), a.get("href", "")
        if t and h:
            candidates.append((t, h))

    # 신형 섹션 UI (/section/101)
    for a in soup.select("div.sa_text a.sa_text_title"):
        t, h = a.get_text(strip=True), a.get("href", "")
        if t and h:
            candidates.append((t, h))

    # 기타 변형
    for a in soup.select("a.sh_text_headline, a.cluster_head_link, a.sa_item_title"):
        t, h = a.get_text(strip=True), a.get("href", "")
        if t and h:
            candidates.append((t, h))

    # 모바일 섹션 폴백
    for a in soup.select("div.press_story a, div.section_list a"):
        t, h = a.get_text(strip=True), a.get("href", "")
        if t and h and "news.naver.com" in h:
            candidates.append((t, h))

    # 중복 제거
    seen, uniq = set(), []
    for t, h in candidates:
        key = (t, h)
        if key not in seen:
            seen.add(key)
            uniq.append((t, h))
    return uniq

def fetch_economy_headlines(limit=NEWS_LIMIT):
    urls = [
        "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101",
        "https://news.naver.com/section/101",
        "https://m.news.naver.com/section/101",
    ]
    for url in urls:
        try:
            r = requests.get(url, headers=HEADERS, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            arts = _extract_articles(soup)
            if arts:
                return arts[:limit]
        except Exception:
            continue

    # 최후 폴백: RSS
    try:
        rss = requests.get("https://news.naver.com/rss/section/101.xml", headers=HEADERS, timeout=10)
        rss.raise_for_status()
        soup = BeautifulSoup(rss.text, "xml")
        items = []
        for it in soup.select("item")[:limit]:
            title = it.title.get_text(strip=True)
            link = it.link.get_text(strip=True)
            items.append((title, link))
        if items:
            return items
    except Exception:
        pass

    return []

# =========================
# 본문 크롤링 (다중 선택자)
# =========================
def fetch_article_text(url, max_chars=800):
    """
    네이버 뉴스 본문을 다양한 선택자로 시도. 길이는 처리속도 위해 컷.
    """
    try:
        res = requests.get(url, headers=HEADERS, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        body_selectors = [
            "div#dic_area",           # 표준(데스크톱)
            "div#newsct_article",     # 신형 템플릿
            "div.article_body",       # 폴백
            "div#articeBody",         # 과거 대체
        ]
        text = ""
        for sel in body_selectors:
            node = soup.select_one(sel)
            if node:
                text = node.get_text(" ", strip=True)
                if text:
                    break
        return text[:max_chars] if text else ""
    except Exception:
        return ""

# =========================
# KoBART 요약 (최초 1회 로딩)
# =========================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("🔄 KoBART 모델 로딩 중...")
TOKENIZER = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
MODEL = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization").to(DEVICE)
print("✅ KoBART 모델 로딩 완료")

def kobart_summarize(text: str, max_len=SUMMARY_MAXLEN) -> str:
    if not text or len(text) < 50:
        return "본문이 짧아 요약 불가"
    try:
        inputs = TOKENIZER([text], max_length=1024, truncation=True, return_tensors="pt").to(DEVICE)
        summary_ids = MODEL.generate(
            inputs["input_ids"],
            num_beams=2,            # 속도 최적화 (품질↑면 3~4)
            max_length=max_len,     # 요약 길이
            early_stopping=True
        )
        return TOKENIZER.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        return f"(요약 실패: {e})"

# =========================
# 최종 요약 메시지 생성
# =========================
def build_news_message(limit=NEWS_LIMIT) -> str:
    lines = []
    kst = pytz.timezone("Asia/Seoul")
    now_str = datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S")
    lines.append(f"⏰ {now_str} 기준\n")
    lines.append("📊 오늘의 경제 뉴스 요약 (네이버)\n")

    articles = fetch_economy_headlines(limit=limit)
    if not articles:
        lines.append("⚠️ 최신 기사 목록을 가져오지 못했습니다. 잠시 후 다시 시도할게요.")
        return "\n".join(lines).strip()

    for i, (title, link) in enumerate(articles, 1):
        body = fetch_article_text(link, max_chars=800)
        summary = kobart_summarize(body, max_len=SUMMARY_MAXLEN)
        lines.append(f"{i}) {title}\n🔗 {link}\n📝 {summary}\n")

    return "\n".join(lines).strip()

# =========================
# 스케줄 작업
# =========================
def job():
    try:
        msg = build_news_message(limit=NEWS_LIMIT)
        resp = send_telegram_message(msg)
        print("텔레그램 전송 결과:", resp)
    except Exception as e:
        print("전송 오류:", e)

# 매일 KST 기준 SEND_TIME에 실행
schedule.every().day.at(SEND_TIME).do(job)
print(f"스케줄러 시작됨: 매일 {SEND_TIME} (KST)에 네이버 경제 뉴스 {NEWS_LIMIT}개 KoBART 요약 전송")

# 즉시 한 번 테스트 전송하고 싶으면 주석 해제:
# job()

# 루프
while True:
    schedule.run_pending()
    time.sleep(1)
